Importamos las librerìas necesarias

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# Version 1

In [ ]:
def scrape_books_list(url, num_books):
    # URL base de la página
    base_url = "https://ww3.lectulandia.com"
    page_num = 1
    books_data = []

    while len(books_data) < num_books:
        # URL de la página actual
        current_url = f"{base_url}/book/page/{page_num}/"

        response = requests.get(current_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todos los elementos que contienen información de los libros
        book_elements = soup.find_all('article', class_='card')

        # Iterar sobre los elementos de los libros
        for book in book_elements:
            if len(books_data) >= num_books:
                break

            # Obtener el título del libro
            title = book.find('a', class_='title').text.strip()

            # Obtener el autor del libro
            author = book.find('div', class_='subdetail').a.text.strip()

            # Obtener la URL del libro
            book_url = base_url + book.find('a', class_='title')['href']

            # Realizar la solicitud GET a la página del libro para obtener el género
            book_response = requests.get(book_url)
            book_soup = BeautifulSoup(book_response.text, 'html.parser')

            # Encontrar el elemento que contiene el género del libro
            genre_elements = book_soup.find('div', id='genero')

            # Encontrar todos los enlaces <a> dentro del elemento
            genre_links = genre_elements.find_all('a') if genre_elements else []

            # Obtener el o los género/s del libro
            genres = ", ".join([link.text.strip() for link in genre_links])

            # Obtener la descripción del libro
            description_element = book.find('div', class_='description')
            description = description_element.p.text.strip() if description_element else "Sin descripción"

            # Agregar los datos del libro a la lista
            books_data.append([title, author, description, genres])

        # Incrementar el número de página
        page_num += 1

        # Crear un DataFrame a partir de la lista de libros
        books_df = pd.DataFrame(books_data, columns=['Title', 'Author', 'Description', 'Genres'])

    return books_df

In [ ]:
# Llamada a la función
url = "https://ww3.lectulandia.com/"
books = scrape_books_list(url, 500)

In [ ]:
books

,Title,Author,Description,Genres
0,Sueñan con ser como nosotras,Jessica Goodman,"Carismáticas, intocables, destinadas a triunfa...","Intriga, Juvenil, Novela"
1,El descontento,Beatriz Serrano,"El descontento es la historia de Marisa, una m...","Novela, Realista"
2,Palabra de reina,Gema Bonnín,La novela sobre la vida y el poder de Catalina...,"Histórico, Novela"
3,Irresistible,Jose de la Rosa,"Adam Baxley, el disoluto heredero del conde de...","Histórico, Novela, Romántico"
4,La criatura del deseo,Andrea Camilleri,"Basándose en personajes y hechos reales, Andre...","Drama, Histórico, Novela"
...,...,...,...,...
495,Roxy,Jarrod Shusterman,Falta poco para que terminen de construir la a...,"Drama, Novela"
496,Hijos de Gael,Rodrigo Costoya,"En la guerra de la razón contra el dogma, un n...","Histórico, Novela"
497,Calamar : casi película policiaca en tres jorn...,Pedro Muñoz Seca,Calamar es una comedia teatral del autor Pedro...,"Humor, Teatro"
498,Esos besos robados son míos,Marian Arpa,Grace se cruza en la vida de Hans en el peor m...,"Erótico, Humor, Novela, Romántico"


In [ ]:
df['Combined'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Genres']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Función para obtener recomendaciones directas basadas en la respuesta del usuario
def direct_recommendation(df, user_input):
    # Combinar título, descripción y géneros para mejorar la clasificación
    df['Combined'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Genres']

    # Vectorización del texto
    vectorizer = TfidfVectorizer(stop_words='spanish')
    tfidf_matrix = vectorizer.fit_transform(df['Combined'])

    # Vectorización de la entrada del usuario
    user_input_tfidf = vectorizer.transform([user_input])

    # Calcular la similitud del coseno entre la entrada del usuario y el conjunto de libros
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix).flatten()

    # Obtener los índices de los libros con mayor similitud
    related_docs_indices = cosine_similarities.argsort()[-3:][::-1]

    # Obtener los libros recomendados
    recommendations = df.iloc[related_docs_indices]

    return recommendations

def display_recommendations(recommendations):
    if recommendations.empty:
        print("No se encontraron libros que coincidan con tu búsqueda.")
        return

    print("Recomendaciones basadas en tus intereses:")
    for index, row in recommendations.iterrows():
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

# Función principal para interactuar con el usuario
def main():
    while True:
        print("\nOpciones de recomendación:")
        print("1. Recomendación Directa")
        print("2. Elección por Autor")
        print("3. Salir")

        choice = input("Selecciona una opción (1/2/3): ")

        if choice == '1':
            user_input = input("¿Qué tienes ganas de leer hoy? ")
            recommendations = direct_recommendation(books, user_input)
            display_recommendations(recommendations)
        elif choice == '2':
            author_name = input("Ingrese el nombre del autor: ")
            recommendations = search_by_author(books, author_name)
            display_recommendations(recommendations)
        elif choice == '3':
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, selecciona 1, 2 o 3.")


In [ ]:
import pandas as pd
import numpy as np

def search_by_author(df, author_name):
    # Filtrar libros por el autor especificado
    author_books = df[df['Author'].str.contains(author_name, case=False, na=False)]

    if author_books.empty:
        print("No se encontraron libros del autor especificado.")
        return None

    # Si hay más de un libro, aleatorizar y seleccionar los más relevantes
    if len(author_books) > 1:
        author_books = author_books.sample(frac=1).reset_index(drop=True)  # Aleatorizar

    # Seleccionar los dos primeros libros como los más relevantes
    first_recommendation = author_books.iloc[0:2]

    # Si hay más de dos libros, seleccionar uno más del segundo grupo más relevante
    second_recommendation = None
    if len(author_books) > 2:
        second_recommendation = author_books.iloc[2]

    # Preparar las recomendaciones
    recommendations = {
        'first': first_recommendation,
        'second': second_recommendation
    }

    return recommendations

def display_recommendations(recommendations):
    if not recommendations:
        return

    print("Recomendaciones basadas en el autor especificado:")
    print("\nPrimer grupo de recomendaciones:")
    for index, row in recommendations['first'].iterrows():
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

    if recommendations['second'] is not None:
        print("\nSegunda recomendación:")
        row = recommendations['second']
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

# Ejemplo de uso:
author_name = input("Ingrese el nombre del autor: ")
recommendations = search_by_author(books, author_name)
display_recommendations(recommendations)


Ingrese el nombre del autor: Jose de la Rosa
Recomendaciones basadas en el autor especificado:

Primer grupo de recomendaciones:
Título: Irresistible
Autor: Jose de la Rosa
Descripción: Adam Baxley, el disoluto heredero del conde de Dunwich, se encuentra en una encrucijada desesperada. Su padre le impone un matrimonio y un heredero en un plazo implacable. Si fracasa, perderá su posición, fortuna y título, cayendo en la desgracia social. La elegida para ser su esposa no puede ser peor, pues se trata de […]
Géneros: Histórico, Novela, Romántico
----------------------------------------


In [ ]:
# Funcion de similitud de coseno
def cosine_similarity(A, B):
    """
    Calcula la similitud del coseno entre dos vectores A y B.

    Parámetros:
    - A, B: Vectores de entrada.

    Retorna:
    - Similitud del coseno entre A y B.
    """
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    return dot_product / (norm_A * norm_B)

# Ejemplo de uso:
vector_A = np.array([1, 2, 3])
vector_B = np.array([4, 5, 6])

print(cosine_similarity(vector_A, vector_B))

In [ ]:
## O PODRIA SER ELMO TAMBIEN
## ESTO LO COPIE DE LA TEORIA

!pip install elmoformanylangs
# Descarga modelo en español
!wget http://vectors.nlpl.eu/repository/11/145.zip
!unzip 145.zip -d elmo_es
# Fix para evitar error en Colab (https://github.com/HIT-SCIR/ELMoForManyLangs/issues/100)
!pip uninstall overrides
!pip install overrides==3.1.0

# Cargamos el modelo desde la carpeta donde fue descomprimido:
from elmoformanylangs import Embedder

# Ruta al directorio del modelo descargado
model_dir = '/content/elmo_es'

# Inicializar el modelo
e = Embedder(model_dir)

# Y aquí probaremos como ELMo nos permite obtener diferentes embeddings según el contexto:
# Lista de frases que contienen la palabra "banco" en diferentes contextos
sents = [
    ['Fui', 'al', 'banco', 'a', 'retirar', 'dinero'],  # Contexto financiero
    ['Me', 'senté', 'en', 'el', 'banco', 'del', 'parque'],  # Contexto de asiento
    ['El', 'jugador', 'estuvo', 'en', 'el', 'banco', 'de', 'suplentes']  # Contexto deportivo
]

# Le ponemos nombres a los contextos, para luego identificarlos
contexts = ['financiero', 'asiento', 'deportivo']

# Obtener embeddings
embeddings = e.sents2elmo(sents)

# Imprimir los embeddings para la palabra "banco" en cada contexto
for idx, (sentence, embedding) in enumerate(zip(sents, embeddings)):
    # El embedding de la palabra "banco" se encuentra en la posición donde aparece en cada frase.
    position = sentence.index('banco')

    # Obtener el embedding de la palabra "banco" en esa posición
    banco_embedding = embedding[position]

    # Determinar el contexto basado en la frase
    context = contexts[idx]

    print(f"Embedding para 'banco' en contexto '{context}':\n{banco_embedding}\n")

## ESTO ME LO TIRA CHATGPT
pip install tensorflow tensorflow_hub scikit-learn pandas numpy
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

# Función para obtener las representaciones de ELMo
def embed_text(text_list):
    embed = hub.load("https://tfhub.dev/google/elmo/3")
    embeddings = embed(text_list, signature="default", as_dict=True)["default"]
    return embeddings

# Función para obtener recomendaciones directas basadas en la respuesta del usuario
def direct_recommendation(df, user_input):
    # Combinar título, descripción y géneros para mejorar la clasificación
    df['Combined'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Genres']

    # Obtener las representaciones de ELMo para los textos combinados de los libros
    combined_texts = df['Combined'].tolist()
    book_embeddings = embed_text(combined_texts).numpy()

    # Obtener la representación de ELMo para la entrada del usuario
    user_input_embedding = embed_text([user_input]).numpy()

    # Calcular la similitud del coseno entre la entrada del usuario y el conjunto de libros
    cosine_similarities = cosine_similarity(user_input_embedding, book_embeddings).flatten()

    # Obtener los índices de los libros con mayor similitud
    related_docs_indices = cosine_similarities.argsort()[-3:][::-1]

    # Obtener los libros recomendados
    recommendations = df.iloc[related_docs_indices]

    return recommendations

def display_recommendations(recommendations):
    if recommendations.empty:
        print("No se encontraron libros que coincidan con tu búsqueda.")
        return

    print("Recomendaciones basadas en tus intereses:")
    for index, row in recommendations.iterrows():
        print(f"Título: {row['Title']}")
        print(f"Autor: {row['Author']}")
        print(f"Descripción: {row['Description']}")
        print(f"Géneros: {row['Genres']}")
        print("-" * 40)

# Función principal para interactuar con el usuario
def main():
    while True:
        print("\nOpciones de recomendación:")
        print("1. Recomendación Directa")
        print("2. Elección por Autor")
        print("3. Salir")

        choice = input("Selecciona una opción (1/2/3): ")

        if choice == '1':
            user_input = input("¿Qué tienes ganas de leer hoy? ")
            recommendations = direct_recommendation(books_df, user_input)
            display_recommendations(recommendations)
        elif choice == '2':
            author_name = input("Ingrese el nombre del autor: ")
            recommendations = search_by_author(books_df, author_name)
            display_recommendations(recommendations)
        elif choice == '3':
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, selecciona 1, 2 o 3.")

# Ejemplo de uso
if __name__ == "__main__":
    # Cargar los datos (esto es solo un ejemplo, asegúrate de cargar tu DataFrame correctamente)
    # books_df = pd.read_csv('path_to_your_books_data.csv') # Cargar el DataFrame desde un archivo CSV

    # Para fines de demostración, aquí hay un ejemplo de un DataFrame simulado:
    books_data = {
        'Title': ['Libro1', 'Libro2', 'Libro3'],
        'Author': ['Autor1', 'Autor2', 'Autor3'],
        'Description': ['Descripción1', 'Descripción2', 'Descripción3'],
        'Genres': ['Género1', 'Género2', 'Género3']
    }
    books_df = pd.DataFrame(books_data)

    main()


# Version 2

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Crear un DataFrame vacío con las columnas especificadas
df_libros = pd.DataFrame(columns=['Genero', 'Titulo', 'Autor', 'Descripcion'])

# Obtener el contenido HTML de la página principal
url_index = 'https://ww3.lectulandia.com'
response_index = requests.get(url_index)
html_content_index = response_index.text
soup_index = BeautifulSoup(html_content_index, 'html.parser')

# Armamos un objeto vacio donde vamos a ir almacenando los libros
data = {'Genero': [],
        'Titulo': [],
        'Autor': [],
        'Descripcion': []}

# Lista de generos a buscar
generos = ["ficcion", "historia", "misterio", "infantil", "arte", "economia", "deporte", "poesia", "religion", "terror"]
# generos = ["ficcion"]

# Encontramos el tag <section> con id 'secgenero' que es el que contiene a todos los generos
soup_generos = soup_index.find('section', id='secgenero')

# Iteramos la lista de generos, y obetenemos el enlace correspondiente
for genero in generos:
  # Buscamos el tag <a> que en e href contiene la url correspondiente
  soup = soup_generos.find('a', href=f'/genero/{genero}/')
  count_genero = 0
  if soup:
    # Construir el enlace completo y armamos un nuevo objeto soup
    url_genero = url_index + soup["href"]
    response_genero = requests.get(url_genero)
    html_content_genero = response_genero.text
    soup_genero = BeautifulSoup(html_content_genero, 'html.parser')
    # Encuentra todas las etiquetas <article> con la clase "card"
    lista_articulos = soup_genero.find_all('article', class_='card')
    for articulo in lista_articulos:
      if count_genero == 10:
        break
      else:
        enlace_card = articulo.find('a', class_='card-click-target')
        href_value = enlace_card.get('href')
        url_libro = url_index + href_value
        response_libro = requests.get(url_libro)
        html_content_libro = response_libro.text
        soup_libro = BeautifulSoup(html_content_libro, 'html.parser')

        titulo = soup_libro.find('div', id='title').find('h1').text.strip()
        if titulo in data['Titulo']:
          continue
        else:
          data['Titulo'].append(titulo)
          data['Genero'].append(genero)
          autor = soup_libro.find('div', id='autor').find('a').text.strip()
          data['Autor'].append(titulo)
          desc = soup_libro.find('div', id='sinopsis').get_text(strip=True)
          data['Descripcion'].append(desc)
          count_genero += 1

df = pd.DataFrame(data)

In [2]:
df_libros = df.copy()

### Tokenizar

In [3]:
import re
import unicodedata

def preprocesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar acentos
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

    # Eliminar puntuación
    texto = re.sub(r'[^\w\s]', '', texto)

    return texto

In [4]:
from transformers import BertTokenizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from gensim.models import Word2Vec

# Función para tokenizar el texto utilizando BERT
def tokenizar_con_bert(texto):
  # Cargar el tokenizador BERT pre-entrenado en español
    tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
    return tokenizer.tokenize(texto)

# Función para tokenizar las sinopsis
def tokenizar_con_wordtokenizer(texto):
    return word_tokenize(texto.lower())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# PRUEBA

In [6]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [7]:
# Instalar la librería necesaria
# !pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Función para preprocesar texto
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\sáéíóúüñ]', '', text)
    # Tokenización
    tokens = word_tokenize(text, language='spanish')
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Reconstruir el texto preprocesado
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

# Función para obtener embeddings de oraciones
def get_sentence_embeddings(sentences):
    model = SentenceTransformer('msmarco-MiniLM-L-6-v3')
    embeddings = model.encode(sentences)
    return embeddings

# Función para calcular similitud entre una consulta y descripciones
def calculate_similarity(user_query, df, top_n=3):
    # Preprocesar la consulta del usuario
    preprocessed_query = preprocess_text(user_query)

    # Obtener embedding de la consulta
    query_embedding = get_sentence_embeddings([preprocessed_query])[0]

    # Calcular similitud coseno entre la consulta y las descripciones
    similarities = df['embedding'].apply(lambda x: util.cos_sim(query_embedding, x))

    # Seleccionar los índices de los libros con mayor similitud
    top_indices = similarities.argsort()[-top_n:][::-1]

    # Obtener los detalles de los libros con mayor similitud
    top_recommendations = df.iloc[top_indices]

    return top_recommendations[['Titulo', 'Autor', 'Genero', 'Descripcion']]

In [9]:
# Descargar las stopwords de NLTK
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
# Preprocesar las descripciones y obtener embeddings
preprocessed_descriptions = [preprocess_text(desc) for desc in df_libros['Descripcion']]
embeddings = get_sentence_embeddings(preprocessed_descriptions)
df_libros['embedding'] = embeddings.tolist()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# Solicitar la consulta del usuario
user_query = input("Ingresa tu consulta: ")

# Calcular similitud y obtener recomendaciones
recommended_books = calculate_similarity(user_query, df_libros)

# # Imprimir recomendaciones
# print("Recomendaciones de libros:")
# for i, row in recommended_books.iterrows():
#     print(f"Título: {row['titulo']} - Autor: {row['autor']} - Género: {row['genero']} - Similitud: {row['similarity']:.2f}")
#     print(f"Descripción: {row['descripcion']}\n")

Ingresa tu consulta: vivir en el exterior


In [17]:
recommended_books

,Titulo,Autor,Genero,Descripcion
44,El temperamento y la naturaleza. Escritos sobr...,El temperamento y la naturaleza. Escritos sobr...,arte,Émile Zola (1840-1902) es el novelista más imp...
1,La noche de Walburga,La noche de Walburga,ficcion,1914. La decadente Praga del imperio austrohún...
7,El hombre que perseguía su sombra,El hombre que perseguía su sombra,ficcion,Lisbeth Salander está cumpliendo condena en la...


In [14]:
df_libros

,Genero,Titulo,Autor,Descripcion,embedding
0,ficcion,Carta al padre,Carta al padre,He aquí el inicio de una carta que constituye ...,"[-0.26953911781311035, 0.10464365780353546, -0..."
1,ficcion,La noche de Walburga,La noche de Walburga,1914. La decadente Praga del imperio austrohún...,"[-0.0974372923374176, 0.5612287521362305, 0.08..."
2,ficcion,El dinosaurio,El dinosaurio,"Microrrelato del autor hondureño, uno de los m...","[0.4145367741584778, 0.20522567629814148, 0.26..."
3,ficcion,Capitanes intrépidos,Capitanes intrépidos,Escrita durante su periodo de residencia en Es...,"[-0.024967188015580177, -0.05517301708459854, ..."
4,ficcion,La casa solariega,La casa solariega,EnLa casa solariega(también conocida comoLa fa...,"[0.08995004743337631, 0.36026108264923096, -0...."
...,...,...,...,...,...
95,terror,"Zothique, el último continente","Zothique, el último continente",Junto con sus colegas y amigos epistolares H.P...,"[0.12797874212265015, 0.00958973728120327, 0.0..."
96,terror,Eco,Eco,Hay quien dice que no hay nada peor que perder...,"[0.11368126422166824, -0.14356832206249237, -0..."
97,terror,Mystic Topaz,Mystic Topaz,Cuando abrimos un libro de Pilar Pedraza a men...,"[-0.13067282736301422, -0.06040621176362038, -..."
98,terror,Corazón caníbal,Corazón caníbal,"Claudia Castañeda, joven mexicana en Estados U...","[0.019904378801584244, -0.19625817239284515, -..."


In [15]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer
from gensim.models import Word2Vec

# Tokenizar y preprocesar las descripciones de los libros
# df_libros['Tokens'] = df_libros['Descripcion'].apply(tokenizar_con_bert)
textos_tokenizados = [tokenizar_con_bert(dec) for dec in df_libros['Descripcion']]
# print(textos_tokenizados)
# # Crear objetos TaggedDocument para cada descripción de libro
# documentos = [TaggedDocument(words=token, tags=[i]) for i, token in enumerate(df_libros['Tokens'])]
# print(documentos)
# Entrenar el modelo Doc2Vec
# modelo_doc2vec = Doc2Vec(vector_size=300, window=5, min_count=1, workers=4)
modelo_doc2vec = Doc2Vec(vector_size=20, window=2, min_count=1, workers=4, epochs=1000)
modelo_doc2vec = Doc2Vec(sentences=textos_tokenizados, vector_size=100, window=5, min_count=1, workers=4)
modelo_doc2vec.build_vocab(textos_tokenizados)
modelo_doc2vec.train(textos_tokenizados, total_examples=modelo_doc2vec.corpus_count, epochs=100)

# # Obtener los vectores de documento para cada descripción de libro
# vectores = [modelo_doc2vec.infer_vector(token) for token in df_libros['Tokens']]

# # Agregar las nuevas columnas al DataFrame
# df_libros['Vectores'] = vectores


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

TypeError: gensim.models.word2vec.Word2Vec.__init__() got multiple values for keyword argument 'sentences'

In [ ]:
def recomendacion_directa_word2vec(entrada_usuario, df, modelo_word2vec, num_recomendaciones=3):
    # Preprocesar la entrada del usuario
    entrada_procesada = preprocesar_texto(entrada_usuario)

    # Calcular la similitud entre la entrada del usuario y las sinopsis de los libros usando Word2Vec
    similitudes = {}
    for idx, row in df.iterrows():
        similitudes[idx] = modelo_word2vec.wv.n_similarity(entrada_procesada.split(), row['Descripcion'].split())

    # Ordenar por similitud descendente
    similitudes_ordenadas = sorted(similitudes.items(), key=lambda x: x[1], reverse=True)
    libros_relevantes = df.iloc[[idx for idx, _ in similitudes_ordenadas[:num_recomendaciones]]]

    return libros_relevantes

In [ ]:
recomendacion_directa_word2vec("sangre ficcion miedo", df_libros, modelo_word2vec)

,Genero,Titulo,Autor,Descripcion,Tokens
58,economia,Monzón: Un viaje por el futuro del Océano Índico,Monzón: Un viaje por el futuro del Océano Índico,El desplazamiento del poder geopolítico hacia ...,"[El, desplazamiento, del, poder, geo, ##pol, #..."
60,deporte,Florentino,Florentino,Una biografía rigurosa para comprender y conoc...,"[Una, biografía, riguros, ##a, para, comprende..."
2,ficcion,El dinosaurio,El dinosaurio,"Microrrelato del autor hondureño, uno de los m...","[Micro, ##r, ##re, ##lato, del, autor, hon, ##..."


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recomendar_libros(df, texto_usuario):
    # Procesar la entrada del usuario
    palabras_clave_usuario = tokenizar_con_bert(texto_usuario)
    vector_palabras_clave = modelo_doc2vec.infer_vector(palabras_clave_usuario)

    # Calcular la similitud entre las palabras clave del usuario y las descripciones de los libros
    similitudes = []
    for i, vector_libro in enumerate(df['Vectores']):
        similitud = cosine_similarity([vector_palabras_clave], [vector_libro])[0][0]
        similitudes.append((i, similitud))

    # Ordenar los libros por similitud
    similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)

    # Seleccionar los tres libros más relevantes
    libros_relevantes = similitudes[:3]

    # Mostrar recomendaciones de libros
    print("Recomendaciones de libros:")
    for idx, similitud in libros_relevantes:
        titulo = df.loc[idx, 'Titulo']
        autor = df.loc[idx, 'Autor']
        genero = df.loc[idx, 'Genero']
        reseña = df.loc[idx, 'Descripcion']

        print(f"Título: {titulo}")
        print(f"Autor: {autor}")
        print(f"Género: {genero}")
        print(f"Reseña: {reseña}")
        print()

In [ ]:
# Capturar la entrada del usuario
texto_usuario = input("¿Qué tienes ganas de leer hoy? ")

# Recomendar libros basados en la entrada del usuario
recomendar_libros(df_libros, texto_usuario)

¿Qué tienes ganas de leer hoy? libros infantiles didacticos
Recomendaciones de libros:
Título: A la Esperanza
Autor: A la Esperanza
Género: poesia
Reseña: «To Hope» apareció enPoems, el primer libro de John Keats, publicado en marzo de 1817, pero la composición data de febrero de 1815, cuando el autor contaba con veinte años.IMPORTANTE:Recomendamos descargar los libros de poesía en formato EPUB dado que en PDF pueden tener problemas de visualización.

Título: Mientras el mundo dice no
Autor: Mientras el mundo dice no
Género: poesia
Reseña: Víctor del Árbol siempre ha practicado la escritura poética, sin darle difusión, como una emoción privada, y gracias a este, su primer libro de poemas, descubrimos una palabra clara y directa para abordar tanto los pequeños como los grandes temas de la vida (el amor, la infancia, la pérdida...), sentimientos y emociones de todo calibre, que se van dando con el paso de los años, a través de una sensibilidad y hondura que nos interpelan y nos retratan.